<table style="width: 100%; border-collapse: collapse;" border="0">
<tr>
<td><b>Created:</b> Monday 30 January 2017</td>
<td style="text-align: right;"><a href="https://www.github.com/rhyswhitley/fire_limitation">github.com/rhyswhitley/fire_limitation</td>
</tr>
</table>

<div>
<center>
<font face="Times">
<br>
<h1>Quantifying the uncertainity of a global fire limitation model using Bayesian inference</h1>
<h2>Part 1: Staging data for analysis</h2>
<br>
<br>
<sup>1,* </sup>Douglas Kelley, 
<sup>2 </sup>Ioannis Bistinas, 
<sup>3, 4 </sup>Chantelle Burton, 
<sup>1 </sup>Tobias Marthews, 
<sup>5 </sup>Rhys Whitley
<br>
<br>
<br>
<sup>1 </sup>Centre for Ecology and Hydrology, Maclean Building, Crowmarsh Gifford, Wallingford, Oxfordshire, United Kingdom
<br>
<sup>2 </sup>Vrije Universiteit Amsterdam, Faculty of Earth and Life Sciences, Amsterdam, Netherlands
<br>
<sup>3 </sup>Met Office United Kingdom, Exeter, United Kingdom
<br>
<sup>4 </sup>Geography, University of Exeter, Exeter, United Kingdom
<br>
<sup>5 </sup>Natural Perils Pricing, Commercial & Consumer Portfolio & Pricing, Suncorp Group, Sydney, Australia
<br>
<br>
<h3>Summary</h3>
<hr>
<p> 
This notebook aims to process the separate netCDF4 files for the model drivers (X<sub>i=1, 2, ... M</sub>) and model target (Y) into a unified tabular data frame, exported as a compressed comma separated value (CSV) file. This file is subsequently used in the Bayesian inference study that forms the second notebook in this experiment. The advantage of the pre-processing the data separately to the analysis allows for it be quickly staged on demand. Of course other file formats may be more advantageous for greater compression (e.g. SQLite3 database file).
</p>
<br>
<b>You will need to run this notebook to prepare the dataest before you attempt the Bayesian analysis in Part 2</b>.
<br>
<br>
<br>
<i>Python code and calculations below</i>
<br>
<hr>
</font>
</center>
</div>

## Load libraries

Changed

In [17]:
# data munging and analytical libraries 
import re
import os
import numpy as np
import pandas as pd
from netCDF4 import Dataset 

# graphical libraries
import matplotlib.pyplot as plt
%matplotlib inline

# set paths
inPath = "../outputs/Australia_region/"
outPath = "../data/AUS_inference_data"

nmonths_import = 15*12

## Import and clean data

Set the directory path and look for all netcdf files that correspond to the model drivers and target.

In [18]:
driver_paths_all = [os.path.join(dp, f) for (dp, _, fn) in os.walk(inPath) for f in fn if f.endswith('.nc')]
driver_paths_all = [path for path in driver_paths_all if 'full_period' not in  path]
driver_names_all = [re.search('^[a-zA-Z_]*', os.path.basename(fp)).group(0) for fp in driver_paths_all]

file_table = pd.DataFrame({'filepath': driver_paths_all, 'file_name': driver_names_all})

def checkFilename(driver_name, sat):
    
    if "firecount" in driver_name:
        if sat in driver_name: return True
        return False
    
    return True

#def select_files_for_sat(sat):
#    test = [checkFilename(driver_name, sat) for driver_name in driver_names_all]
    
driver_paths = np.array(driver_paths_all)#test]
driver_names = np.array(driver_names_all)#[test]
print(driver_paths)
#    driver_names = ["firecount" if "firecount" in driver_name else driver_name for driver_name in driver_names]
#    return driver_paths, driver_names
    
driver_info = [driver_paths, driver_names]#[select_files_for_sat(sat) for sat in satalites]

['../outputs/Australia_region/burnt_area-GFED4s_2.5degree_2001-2016.nc'
 '../outputs/Australia_region/firecount-SE_Aus_2001_onwards.nc'
 '../outputs/Australia_region/climate/from_2001/air2001-2019.nc'
 '../outputs/Australia_region/climate/from_2001/emc-2001-2019.nc'
 '../outputs/Australia_region/climate/from_2001/precip2001-2019.nc'
 '../outputs/Australia_region/climate/from_2001/lightning2001-2019.nc'
 '../outputs/Australia_region/climate/from_2001/relative_humidity2001-2019.nc'
 '../outputs/Australia_region/climate/from_2001/soilw.0-10cm.gauss.2001-2019.nc'
 '../outputs/Australia_region/climate/from_2001/swnd.2001-2019.nc'
 '../outputs/Australia_region/climate/from_2001/tmax.2001-2019.nc'
 '../outputs/Australia_region/climate/from_2001/swndMax.2001-2019.nc'
 '../outputs/Australia_region/climate/from_2001/tmaxMax.2001-2019.nc'
 '../outputs/Australia_region/human/from_2001/cropland2001-2019.nc'
 '../outputs/Australia_region/human/from_2001/fract_agr2001-2019.nc'
 '../outputs/Australia_

Define a function to extract the variable values from each netCDF4 file. Variables are flattened from a 3 dimensional array to 1 dimensional version, pooling all values both spatially and temporily. 

Don't know if this is the correct way to do this, but will come back to it once I understand the model (and its optimisation) better.

In [19]:
from pdb import set_trace as browser

def nc_extract(fpath):
    print(fpath)
    print("Processing: {0}".format(fpath))
    with Dataset(fpath, 'r') as nc_file:
        gdata = np.array(nc_file.variables['variable'][:,:,:])
        if (len(gdata) < nmonths_import): return None
        gdata = gdata[13:nmonths_import]
        gdata[gdata < -9E9] = np.nan
        gflat = gdata.flatten()
        if type(gdata) == np.ma.core.MaskedArray:
            return gflat[~gflat.mask].data
        else:
            return gflat.data

Execute the above function on all netCDF4 file paths.

In [20]:
values = [nc_extract(dp) for dp in driver_paths]

../outputs/Australia_region/burnt_area-GFED4s_2.5degree_2001-2016.nc
Processing: ../outputs/Australia_region/burnt_area-GFED4s_2.5degree_2001-2016.nc
../outputs/Australia_region/firecount-SE_Aus_2001_onwards.nc
Processing: ../outputs/Australia_region/firecount-SE_Aus_2001_onwards.nc
../outputs/Australia_region/climate/from_2001/air2001-2019.nc
Processing: ../outputs/Australia_region/climate/from_2001/air2001-2019.nc
../outputs/Australia_region/climate/from_2001/emc-2001-2019.nc
Processing: ../outputs/Australia_region/climate/from_2001/emc-2001-2019.nc
../outputs/Australia_region/climate/from_2001/precip2001-2019.nc
Processing: ../outputs/Australia_region/climate/from_2001/precip2001-2019.nc
../outputs/Australia_region/climate/from_2001/lightning2001-2019.nc
Processing: ../outputs/Australia_region/climate/from_2001/lightning2001-2019.nc
../outputs/Australia_region/climate/from_2001/relative_humidity2001-2019.nc
Processing: ../outputs/Australia_region/climate/from_2001/relative_humidity2

Turn this into a dataframe for the analysis.

In [21]:
for v, n in zip(values, driver_names):
    print(n)
    print(v.shape)

burnt_area
(42084,)
firecount
(42084,)
air
(42084,)
emc
(42084,)
precip
(42084,)
lightning
(42084,)
relative_humidity
(42084,)
soilw
(42084,)
swnd
(42084,)
tmax
(42084,)
swndMax
(42084,)
tmaxMax
(42084,)
cropland
(42084,)
fract_agr
(42084,)
pasture
(42084,)
population_density
(42084,)
MaxOverMean_soilw
(42084,)
MeanAnnaul_soilw
(42084,)
nonetreecover
(42084,)
treecover
(42084,)
vegcover
(42084,)
MeanAnnual_soilw
(42084,)


In [22]:
# turn list into a dataframe
def makeDataFrave(value, driver_names):
    
    df = pd.DataFrame(np.array(value).T, columns=driver_names)
    print(df.info())
    df.dropna(inplace=True)
    return df
fire_df = makeDataFrave(values, driver_names)
#fire_df = [makeDataFrave(value, driver_names[1]) for value, driver_names in zip(values, driver_info)]
#fire_df.info()
# replace null flags with pandas null
#fire_df.replace(fire_df < -3e38, np.nan, inplace=True)
#fire_df[] = np.nan
# drop all null rows (are ocean and not needed in optim)
#fire_df.dropna(inplace=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 42084 entries, 0 to 42083
Data columns (total 22 columns):
burnt_area            18370 non-null float32
firecount             18370 non-null float32
air                   18370 non-null float32
emc                   18370 non-null float32
precip                18370 non-null float32
lightning             18370 non-null float32
relative_humidity     18370 non-null float32
soilw                 18370 non-null float32
swnd                  18370 non-null float32
tmax                  18370 non-null float32
swndMax               18370 non-null float32
tmaxMax               18370 non-null float32
cropland              18370 non-null float32
fract_agr             18370 non-null float32
pasture               18370 non-null float32
population_density    18370 non-null float32
MaxOverMean_soilw     18370 non-null float32
MeanAnnaul_soilw      18370 non-null float32
nonetreecover         18370 non-null float32
treecover             18370 non-null

Check that we've built it correctly.

In [23]:
fire_df.tail(10)

,burnt_area,firecount,air,emc,precip,lightning,relative_humidity,soilw,swnd,tmax,...,cropland,fract_agr,pasture,population_density,MaxOverMean_soilw,MeanAnnaul_soilw,nonetreecover,treecover,vegcover,MeanAnnual_soilw
42014,0.000250,0.079858,17.553213,0.234101,1.660122,55.466782,71.193550,0.217854,8.964027,292.977386,...,0.155878,0.170788,1.490975e-02,0.951528,2.336063,0.212649,0.570938,0.328125,0.899063,0.212649
42022,0.000024,0.000840,22.691923,0.158290,0.141105,28.805443,46.008064,0.031730,8.830502,300.270966,...,0.336812,0.337747,9.351684e-04,10.831301,1.937303,0.143093,0.666188,0.145703,0.811891,0.143093
42023,0.000006,0.015740,25.508051,0.118448,0.242276,65.540100,32.887096,0.000000,7.113547,304.896759,...,0.451122,0.468185,1.706225e-02,24.164396,1.951308,0.155490,0.717969,0.083125,0.801094,0.155490
42024,0.000028,0.006182,26.822567,0.106996,0.514888,99.779335,29.620968,0.000000,6.809575,306.329041,...,0.430092,0.430184,9.220493e-05,1.821281,2.689591,0.044450,0.728516,0.053828,0.782344,0.044450
42025,0.000008,0.014837,25.290310,0.118609,1.167262,346.483307,32.733871,0.000000,7.048820,304.561310,...,0.442137,0.442137,-1.669431e-18,2.547731,2.089751,0.157703,0.791875,0.059687,0.851562,0.157703
42026,0.000019,0.007482,21.980635,0.153859,1.455241,320.945160,43.524193,0.063210,6.395611,301.667755,...,0.632484,0.648351,1.586682e-02,5.697479,1.278537,0.319096,0.813594,0.131797,0.945391,0.319096
42027,0.000040,0.024395,19.730631,0.213798,2.514786,343.224274,65.250000,0.288302,7.548583,297.432220,...,0.282312,0.314917,3.260481e-02,52.900429,1.348908,0.370677,0.581094,0.380625,0.961719,0.370677
42042,0.000194,0.023156,21.371763,0.164333,0.797876,55.290688,47.104839,0.000000,8.355238,299.596771,...,0.528123,0.549115,2.099184e-02,25.089970,3.905432,0.022155,0.767656,0.167253,0.934909,0.022155
42043,0.000111,0.047538,21.024986,0.163345,1.499353,197.752075,46.362904,0.000000,8.645095,298.912933,...,0.441666,0.560192,1.185258e-01,81.966492,3.144927,0.044296,0.694531,0.237734,0.932266,0.044296
42044,0.000131,0.009344,19.873373,0.186224,1.685490,238.623993,54.362904,0.000000,8.260377,297.345154,...,0.194024,0.251290,5.726574e-02,5.663785,1.685585,0.152447,0.467578,0.477539,0.945117,0.152447


Export this to disk to be used by the analysis notebook - used gzip compression to save on space. Beware, because of there are approximation 10 million rows of data, this may take some time.

In [24]:

outPathi = outPath + '.csv'
savepath = os.path.expanduser(outPathi)
fire_df.to_csv(savepath, index=False)

<div>
<br>
<br>
<br>
<center>
<font size="5">
<a style="font-weight: bold; size: 5" href="http://localhost:8888/notebooks/notebooks/bayesian_inference.ipynb">Part 2: click here</a>
</font>
</center>
</div>